In [1]:
# 📦 Import librerie
import pandas as pd
import itertools

# 📥 Caricamento del dataset
df = pd.read_excel("../data/Prodromal_Mondino_Icot.xlsx")

# 🎯 Elenco dei 4 prodromi
prodromi = ['Hyposmia', 'Constipation', 'REM', 'Depression']

# 🧹 Rimuovi righe con prodromi mancanti
df = df.dropna(subset=prodromi)

# 🔄 Converti in int (0 o 1) se non lo sono già
df[prodromi] = df[prodromi].astype(int)

# 🧠 Crea combinazione binaria dei 4 prodromi
df['prodromi_combo'] = df[prodromi].astype(str).agg(''.join, axis=1)

# 🔢 Crea 16 colonne per ogni combinazione (0000 a 1111)
combinazioni_bin = [''.join(c) for c in itertools.product('01', repeat=4)]

for combo in combinazioni_bin:
    df[f'combo_{combo}'] = (df['prodromi_combo'] == combo).astype(int)

# 💾 Salva dataset bilanciabile
df.to_csv("../data/Prodromal_Mondino_Icot_Balanced.csv", index=False)

# ✅ Output diagnostico
print("✅ Dataset salvato come 'Prodromal_Mondino_Icot_Balanced.csv'")
print("📊 Distribuzione combinazioni:\n", df['prodromi_combo'].value_counts())

✅ Dataset salvato come 'Prodromal_Mondino_Icot_Balanced.csv'
📊 Distribuzione combinazioni:
 prodromi_combo
0011    46
0111    37
1111    29
0100    20
0010    20
0000    19
1110    19
1101    16
0101    12
0110    12
1011    10
1100    10
1001    10
0001     8
1010     6
1000     2
Name: count, dtype: int64


| Codice | Hyposmia | Constipation | REM | Depression |
| ------ | -------- | ------------ | --- | ---------- |
| 0000   | ❌        | ❌            | ❌   | ❌          |
| 0001   | ❌        | ❌            | ❌   | ✅          |
| 0010   | ❌        | ❌            | ✅   | ❌          |
| 0011   | ❌        | ❌            | ✅   | ✅          |
| 0100   | ❌        | ✅            | ❌   | ❌          |
| 0101   | ❌        | ✅            | ❌   | ✅          |
| 0110   | ❌        | ✅            | ✅   | ❌          |
| 0111   | ❌        | ✅            | ✅   | ✅          |
| 1000   | ✅        | ❌            | ❌   | ❌          |
| 1001   | ✅        | ❌            | ❌   | ✅          |
| 1010   | ✅        | ❌            | ✅   | ❌          |
| 1011   | ✅        | ❌            | ✅   | ✅          |
| 1100   | ✅        | ✅            | ❌   | ❌          |
| 1101   | ✅        | ✅            | ❌   | ✅          |
| 1110   | ✅        | ✅            | ✅   | ❌          |
| 1111   | ✅        | ✅            | ✅   | ✅          |

In [4]:
print(df['prodromi_combo'].value_counts())

prodromi_combo
11      46
111     37
1111    29
100     20
10      20
0       19
1110    19
1101    16
101     12
110     12
1011    10
1100    10
1001    10
1        8
1010     6
1000     2
Name: count, dtype: int64


In [11]:
import pandas as pd
from pandas.api.types import is_numeric_dtype

# 📥 Carica il dataset originale
df = pd.read_csv('../data/Prodromal_Mondino_Icot_Balanced.csv')

# 🧹 Elimina colonne inutili: identificativi, stringhe, date, combo_X
drop_cols = [col for col in df.columns if (
    col.lower() in ['id', 'surname', 'name', 'center', 'evaluation date'] or
    'combo_' in col.lower()
)]
df = df.drop(columns=drop_cols)

# 🔍 Tieni solo colonne numeriche + la colonna target 'prodromi_combo'
numeric_cols = [col for col in df.columns if is_numeric_dtype(df[col])]
if 'prodromi_combo' not in numeric_cols:
    numeric_cols.append('prodromi_combo')

df = df[numeric_cols]

# 📌 Rendi 'prodromi_combo' categorica (obbligatorio per CTGAN)
df['prodromi_combo'] = df['prodromi_combo'].astype(str)

# ✅ Salva dataset pronto per CTGAN
df.to_csv('../data/prodromi_ctgan_ready.csv', index=False)
print("✅ Dataset pronto per CTGAN salvato in 'prodromi_ctgan_ready.csv'")

✅ Dataset pronto per CTGAN salvato in 'prodromi_ctgan_ready.csv'


In [13]:
import pandas as pd
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
from sklearn.utils import shuffle
import warnings

warnings.filterwarnings("ignore")

# 📥 Carica dataset pre-processato
df = pd.read_csv('../data/prodromi_ctgan_ready.csv')

# 📊 Conta occorrenze delle classi
target_col = 'prodromi_combo'
counts = df[target_col].value_counts()

# 🎯 Classi che vogliamo aumentare fino a 40 (escluse quelle ≥ 40)
threshold = 40
classi_da_espandere = counts[(counts < threshold) & (counts >= 10)].index.tolist()
print(f"➡️ Classi da sintetizzare fino a {threshold} osservazioni:", classi_da_espandere)

# 🔧 Parametri CTGAN (ottimizzati da te precedentemente)
CTGAN_PARAMS = dict(
    epochs=1000,
    batch_size=128,
    generator_lr=2e-4,
    discriminator_lr=2e-4,
    embedding_dim=256,
    generator_dim=(128, 128),
    discriminator_dim=(256, 256),
    enforce_min_max_values=True,
    pac=1
)

# 🧬 Generazione dati sintetici
synth_data_list = []

for combo in classi_da_espandere:
    subset = df[df[target_col] == combo].copy()
    n_current = len(subset)
    n_to_generate = threshold - n_current

    print(f"🔄 Classe {combo}: {n_current} → genero {n_to_generate} nuove righe")

    # Metadati per CTGAN
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(subset)
    metadata.update_column(column_name=target_col, sdtype='categorical')

    # Addestramento CTGAN
    synth = CTGANSynthesizer(metadata=metadata, **CTGAN_PARAMS)
    synth.fit(subset)

    # Generazione sintetica
    sampled = synth.sample(num_rows=n_to_generate)
    sampled[target_col] = combo  # assicurati che il target sia assegnato
    synth_data_list.append(sampled)

# 📦 Unisci dati reali + sintetici
df_synthetics = pd.concat(synth_data_list, ignore_index=True)
df_finale = pd.concat([df, df_synthetics], ignore_index=True)
df_finale = shuffle(df_finale, random_state=42).reset_index(drop=True)

# 💾 Salva il nuovo dataset bilanciato
df_finale.to_csv('../data/prodromi_ctgan_balanced.csv', index=False)
print("✅ Dataset bilanciato salvato in 'prodromi_ctgan_balanced.csv'")


➡️ Classi da sintetizzare fino a 40 osservazioni: [111, 1111, 100, 10, 0, 1110, 1101, 101, 110, 1011, 1100, 1001]
🔄 Classe 111: 37 → genero 3 nuove righe
🔄 Classe 1111: 29 → genero 11 nuove righe
🔄 Classe 100: 20 → genero 20 nuove righe
🔄 Classe 10: 20 → genero 20 nuove righe
🔄 Classe 0: 19 → genero 21 nuove righe
🔄 Classe 1110: 19 → genero 21 nuove righe
🔄 Classe 1101: 16 → genero 24 nuove righe
🔄 Classe 101: 12 → genero 28 nuove righe
🔄 Classe 110: 12 → genero 28 nuove righe
🔄 Classe 1011: 10 → genero 30 nuove righe
🔄 Classe 1100: 10 → genero 30 nuove righe
🔄 Classe 1001: 10 → genero 30 nuove righe
✅ Dataset bilanciato salvato in 'prodromi_ctgan_balanced.csv'


In [14]:
import pandas as pd
import numpy as np
from sdmetrics.reports.single_table import QualityReport
from sdv.metadata import SingleTableMetadata
from scipy.stats import ks_2samp
import matplotlib.pyplot as plt
import seaborn as sns

# 📥 Carica dataset originale e bilanciato
df_real = pd.read_csv("../data/prodromi_ctgan_ready.csv")
df_synth = pd.read_csv("../data/prodromi_ctgan_balanced.csv")

# 🎯 Filtra i dati sintetici aggiunti (tutti quelli oltre la lunghezza originale)
df_synth_only = df_synth.iloc[len(df_real):].reset_index(drop=True)

# 🧠 Crea metadata per QualityReport
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=df_synth)

# 🧪 Genera SDV Quality Report
report = QualityReport()
report.generate(real_data=df_real, synthetic_data=df_synth_only, metadata=metadata.to_dict())

print("🔍 SDV Quality Score:", round(report.get_score(), 4))

# 📊 Mostra dettaglio per ogni colonna
details = report.get_details(property_name='Column Shapes')
print("\n📋 SDV Quality per colonna:")
print(details[['Column', 'Score']].sort_values(by='Score'))

# 📈 Visualizza plot opzionale
report.get_visualization(property_name='Column Shapes').show()

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 31/31 [00:00<00:00, 164.09it/s]|
Column Shapes Score: 93.24%

(2/2) Evaluating Column Pair Trends: |██████████| 465/465 [00:00<00:00, 984.61it/s]|
Column Pair Trends Score: 87.29%

Overall Score (Average): 90.26%

🔍 SDV Quality Score: 0.9026

📋 SDV Quality per colonna:
                                               Column     Score
25                                         Gait Speed  0.876534
5                                             %det AP  0.887845
28                                                REM  0.896562
29                                         Depression  0.898823
4                                             %det ML  0.899068
30                                     prodromi_combo  0.903672
3                                              %det V  0.903836
7                                                 Age  0.904945
27                                           Hyposmia  0.910919
0                     

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed